In [1]:
# Import needed libraries
import pandas as pd
import numpy as np 
import math

In [2]:
# Read data
mushroom_data = pd.read_csv('agaricus-lepiota.data', names=['poison-label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'])
mushroom_data = mushroom_data.reindex(np.random.permutation(mushroom_data.index))

# Separate out labels
mushroom_labels = pd.DataFrame(mushroom_data['poison-label'])
mushroom_data.drop('poison-label', axis=1, inplace=True)

In [3]:
# Convert non-numerical data to numbers
def convert_to_numbers(column):
    vals = {}
    currMapIndex = 0
    for item in column: 
        if item not in vals:
            vals[item] = currMapIndex
            currMapIndex += 1
    column.replace(vals, inplace=True)
    
for colName, colData in mushroom_data.iteritems():
    convert_to_numbers(colData)

In [4]:
# Add cross features
mushroom_data['stalk-shape-and-root'] = mushroom_data['stalk-shape'] * mushroom_data['stalk-root']
mushroom_data['veil-type-and-color'] = mushroom_data['veil-type'] * mushroom_data['veil-color']
mushroom_data['all-cap'] = mushroom_data['cap-shape'] * mushroom_data['cap-surface'] * mushroom_data['cap-color']
mushroom_data['all-gill'] =  mushroom_data['gill-size'] * mushroom_data['gill-color'] * mushroom_data['gill-spacing']
mushroom_data['ring-type-and-number'] = mushroom_data['ring-type'] * mushroom_data['ring-number']
mushroom_data['stalk-and-cap'] = mushroom_data['stalk-shape-and-root'] * mushroom_data['all-cap']
mushroom_data['gill-and-veil'] = mushroom_data['all-gill'] * mushroom_data['veil-type-and-color']
mushroom_data['stalk-and-gill'] = mushroom_data['stalk-shape-and-root'] * mushroom_data['all-gill']

In [5]:
# Convert edible/poisonous to boolean 
mushroom_labels.replace({'p': 0, 'e': 1}, inplace=True)

In [6]:
# Separate into train and test sets
mushroom_data_train = mushroom_data.iloc[:7325]
mushroom_data_test = mushroom_data.iloc[7325:]

mushroom_labels_train = mushroom_labels[:7325]
mushroom_labels_test = mushroom_labels[7325:]

In [7]:
# Convert dataframes to numpy arrays
mushroom_data_train = mushroom_data_train.to_numpy().T
mushroom_data_test = mushroom_data_test.to_numpy().T

mushroom_labels_train = mushroom_labels_train.to_numpy().T
mushroom_labels_test = mushroom_labels_test.to_numpy().T

In [8]:
# Initialize parameters
def initialize_parameters(input_shape):
    W = np.random.randn(input_shape[0],1) * .01
    b = np.random.randn() * .01
    
    return (W, b)

In [9]:
# Sigmoid implementation
def sigmoid(z):
    return 1 / ( 1 + np.exp(z * -1))

In [10]:
# Forward propagation implementation
def forward_prop(weights, bias, input_data):
    Z = np.dot(weights.T, input_data) + bias
    return sigmoid(Z)

In [11]:
# Loss function implementation
def compute_loss(y_actual, y_predicted):
    epsilon = 1e-5
    inputSize = y_actual.shape[1]
    
    loss = -1/inputSize * np.sum(y_actual * np.log(y_predicted + epsilon) + (1 - y_actual) * np.log((1 - y_predicted) + epsilon))
    return loss

In [12]:
# Back propagation implementation
def back_prop(input_data, y_actual, y_predicted):
    inputSize = y_actual.shape[1]
    
    dz = y_predicted - y_actual
    db = 1/inputSize * np.sum(dz)
    dw = 1/inputSize * np.dot(input_data, dz.T)
    
    return (dz, db, dw)

In [13]:
# Update weights function implementation
def update_weights(weights, bias, dw, db, learning_rate): 
    weights = weights - learning_rate * dw
    bias = bias - learning_rate * db
    return (weights, bias)

In [14]:
# Function to train model
def train_model(epochs, learning_rate, input_data, input_labels):
    
    # Initialize weights
    W, b = initialize_parameters(input_data.shape)
    
    for i in range(epochs): 
        # Forward propagation
        predictions = forward_prop(W, b, input_data)
        
        # Compute loss
        loss = compute_loss(input_labels, predictions)
        
        if i % 300 == 0:
            print('Loss on epoch', i+1, 'is', loss)
        
        # Back propagation
        dz, db, dw = back_prop(input_data, input_labels, predictions)
        
        # Update weights
        W, b = update_weights(W, b, dw, db, learning_rate)
        
    # Return final weights and bias
    return W, b

In [16]:
# Train model
final_weights, final_bias = train_model(8000, .5, mushroom_data_train, mushroom_labels_train)

Loss on epoch 1 is 0.6798813542869999
Loss on epoch 301 is 0.11405378564848676
Loss on epoch 601 is 0.09793766655060385
Loss on epoch 901 is 0.0887296023655224
Loss on epoch 1201 is 0.08250311742126078
Loss on epoch 1501 is 0.07788203350897029
Loss on epoch 1801 is 0.07423677623762527
Loss on epoch 2101 is 0.07123547343425987
Loss on epoch 2401 is 0.06868584509388395
Loss on epoch 2701 is 0.06646840467312443
Loss on epoch 3001 is 0.06450470745363397
Loss on epoch 3301 is 0.06274090264989579
Loss on epoch 3601 is 0.06113862462068974
Loss on epoch 3901 is 0.05966966988592273
Loss on epoch 4201 is 0.05831274492407674
Loss on epoch 4501 is 0.057051402650620434
Loss on epoch 4801 is 0.05587268954024589
Loss on epoch 5101 is 0.054766232723653066
Loss on epoch 5401 is 0.05372360791401786
Loss on epoch 5701 is 0.05273789141495867
Loss on epoch 6001 is 0.051803335584522624
Loss on epoch 6301 is 0.05091512869215304
Loss on epoch 6601 is 0.05006921335893097
Loss on epoch 6901 is 0.049262146171059

In [17]:
# Function to test model 
def test_model(test_input, test_labels, weights, bias): 
    
    # Get predictions
    predictions = forward_prop(weights, bias, test_input)
    
    # Compute loss
    loss = compute_loss(test_labels, predictions)
    
    print('Loss is', loss)

In [18]:
test_model(mushroom_data_test, mushroom_labels_test, final_weights, final_bias)

Loss is 0.05641289022115028


In [242]:
# Function to predict on new input
def predict(new_input, weights, bias):
    predictions = forward_prop(new_input, weights, bias)
    return predictions